### Download the data

In [ ]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT= "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets","housing")
HOUSING_URL= DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=HOUSING_PATH)
    housing_tgz.close()

### Load the data

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

### Display the data

In [ ]:
fetch_housing_data()
housing = load_housing_data()
housing.head(5)

In [ ]:
housing.info()

In [ ]:
housing["ocean_proximity"].value_counts()

In [ ]:
housing.describe()

### Plot the data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))

### Create a Testset

In [ ]:
import numpy as np

def split_train_test(data, test_ratio):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

### Split by id

In [ ]:
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
housing_with_id = housing.reset_index() # adds an index column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])
housing["income_cat"].hist()

#### With this, we'll stratify the dataset based on "income_cat".

In [ ]:
housing["income_cat"].value_counts() / len(housing) # sum => 1.0

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in strat_split.split(housing, housing["income_cat"]):
    print(f"Train_index {train_index}\nTest_index {test_index}")
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

#### With the test set also Stratified, we should now see the same percentage as we saw above

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set) # sum => 1.0

#### Remove 'income_cat' to bring back dataset to original

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
strat_train_set.describe()

In [ ]:
housing = strat_train_set.copy()
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1) #alpha helps in  seeing the density of the scatter plot

In [ ]:
housing.plot(kind="scatter" ,x="longitude", y="latitude", alpha=0.4, s=housing["population"]/100, label="Population",
figsize=(10,7), c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()

#### Let's calculate coefficient between each feature

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False) # corr values w/respect to median_house_value'

In [ ]:
corr_matrix

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))

In [ ]:
housing.plot(kind="scatter", x="median_house_value", y="median_income", alpha=0.1)

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"] / housing["households"]

corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

# DATA CLEANING for ML ALGO

In [ ]:
# Clean the dataset by filling empty values with their respective feature medians
from sklearn.impute import SimpleImputer

housing_num = housing.drop("ocean_proximity",axis=1) # This needs to be done becase SimpleImputer only computes medians from numerical values and not strings
imputer = SimpleImputer(strategy="median") # Set the strategy to be Median(which fills N/As with medians, IIly there are other strategies that impute N/As)

imputer.fit(housing_num)

print(imputer.statistics_) # will contain medians of all the features
print(housing_num.median().values) # should be same as above

In [ ]:
# The imputer now learned the required values(medians, etc) for our dataset
# We can now apply this to the dataset(Remember to give only numerical values)
x = imputer.transform(housing_num)

In [ ]:
# x is a numpy array with just the data, so let's covert them to DF and add our original headers
housing_tr = pd.DataFrame(x, columns=housing_num.columns, index=housing_num.index)

### Handling Categorical values

In [ ]:
# We have only one Categorial variable - 'ocean_proximity'
housing["ocean_proximity"].value_counts()

In [ ]:
housing_cat = housing[["ocean_proximity"]]

In [ ]:
# Using OrdianlEncoder
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_

In [ ]:
# Though the above approach will not help as it is again categorical and doesn't
# actually represent the feature, therefore we'll use a OneHotEncoder to append binary columns
from sklearn.preprocessing import OneHotEncoder

hot_encoder = OneHotEncoder()
housing_cat_hotcoded = hot_encoder.fit_transform(housing_cat)
housing_cat_hotcoded.toarray()

In [ ]:
hot_encoder.categories_

In [ ]:
# Creating our own Encoder that goes hand-in-hand with SciKitLearn's APIs
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6 # indices of their respective columns from orig dataset

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self, X, y=None):
        return self # nothing to do here

    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [ ]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attr = attr_adder.transform(housing.values) # housing.values gives an np array of the dataset
housing_extra_attr[:5]

## Feature Scaling

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy="median")),
    ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
# Using ColumnTransformer to transforms the dataset together
# instead of doing it separately for Categorical & Numerical features.
from sklearn.compose import ColumnTransformer

num_attributes = list(housing_num)
cat_attributes = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attributes),
    ("cat", OneHotEncoder(), cat_attributes)
])
housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
random_data = housing.iloc[:5]
random_label = housing_labels.iloc[:5]
random_data_prepared = full_pipeline.transform(random_data)
print("Predictions:", lin_reg.predict(random_data_prepared))
print("Original vals:", list(random_label))

#### Find RMSE(Root Mean Squared Error) for the predictions

In [ ]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
mse = mean_squared_error(housing_labels, housing_predictions)
rmse = np.sqrt(mse)
rmse

In [ ]:
housing_labels.describe()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Decision Tree Regressor on full data_set
housing_predictions = tree_reg.predict(housing_prepared)
mse = mean_squared_error(housing_labels, housing_predictions)
rmse = np.sqrt(mse)
rmse

### Cross-Validation

Running RMSE on the one whole training only vaguely shows us over/underfitting of data. Doing a cross-validation, which splits the training set into multiple sets and runs the model on them separately and returns a list of the prediction metrics(mean and S.D)

In [ ]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())

In [ ]:
# Decision Tree Regressor with Cross-Validation
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

In [ ]:
# Linear Regression With Cross-Validation
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
# Random Forest Regressor on full data_set
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
forest_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, forest_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
# Random Forest Regressor with Cross-Validation
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

In [ ]:
# Save the models
import joblib

# Writes the model to disc
joblib.dump(forest_reg, "models/forest_reg.pkl")
joblib.dump(tree_reg, "models/tree_reg.pkl")
joblib.dump(lin_reg, "models/lin_reg.pkl")

# Loads the model from disc
# forest_reg = joblib.load("models/forest_reg.pkl")

#### Fine-Tune the Models

Here, we'll be tuning the model parameters. The parameters will be explored in detail in the upcoming chapters, for now it is okay to go ahead even if you don't understand them.

Hyperparameter tuning can be done manually but will take a lot of time to find a great combination. We can use GridSearchCV and RandomSearchCV for this purpose

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_hhold"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attributes + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse) # => 47,730.2

In [ ]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) -1,loc=squared_errors.mean(), scale=stats.sem(squared_errors)))